<a href="https://colab.research.google.com/github/kikiymini/7506R-1C2024-GRUPO02/blob/main/TP2/7506R_TP2_GRUPO2_CHP1_ENTREGA_N1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Organizaci&oacute;n de Datos</center>

#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 2: Propiedades en venta</center>

### <center> Grupo 2</center>

## Integrantes:

*   Aramayo Carolina
*   Utrera Maximo Damian
*   Villalba Ana Daniela
*   Fiorilo Roy


# Importación de librerias

In [68]:
from google.colab import drive

import pandas as pd

# Tokens y preprocesamiento
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

# nltk
nltk.download('stopwords')
nltk.download('punkt')

# Modelos
import joblib
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Lectura de archivo


### Desde google drive

In [53]:
drive.mount('/content/drive')
drive_path = "/content/drive/MyDrive/7506R-1C2024-GRUPO02/TP2"
train_path = drive_path + '/Dataset/train.csv'
test_path = drive_path + '/Dataset/test.csv'
sample_path = drive_path + '/Dataset/sample_solution.csv'
ds_train = pd.read_csv(train_path)
ds_test = pd.read_csv(test_path)
sample = pd.read_csv(sample_path)
using_drive = True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Desde maquina local

In [ ]:
# Si se esta trabajando en local, descomentar esta celta y comentar la de arriba
# train_path = './Dataset/train.csv'
# test_path = './Dataset/test.csv'
# ds_train = pd.read_csv(train_path)
# ds_test = pd.read_csv(test_path)
# using_drive = False

### Visualizamos los datasets

In [14]:
ds_train

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo
...,...,...,...
49995,49995,Pensé que esta película hizo un buen trabajo a...,positivo
49996,49996,"Mala parcela, mal diálogo, mala actuación, dir...",negativo
49997,49997,Soy católica enseñada en escuelas primarias pa...,negativo
49998,49998,Voy a tener que estar en desacuerdo con el com...,negativo


Dividimos el dataset de train en x_train e y_train (la primera tendra el texto y la segunda las labels).

Adicionalmente no incluiremos la columna ID ya que no es necesaria para nuestro proposito de entrenamiento.

In [34]:
x_train_text = ds_train["review_es"]
y_train = ds_train["sentimiento"]

In [12]:
ds_test

,ID,review_es
0,60000,La mayor virtud de esta película es su existen...
1,60001,"No soy un experto cinéfilo, pero pocas veces m..."
2,60002,Si no eres un incondicional del humor estilo T...
3,60003,"No sé qué está pasando, si la gente se deja ll..."
4,60004,"Pero cuando amanece,y me quedo solo,siento en ..."
...,...,...
8594,68594,"Buena no, lo siguiente. Por fin un film serio ..."
8595,68595,"Me esperaba mucho, pero que mucho, más.Guión m..."
8596,68596,"De mal cuerpo como sensación al finalizar, de ..."
8597,68597,Los que han añadido comentarios os lo han dich...


Creamos el dataset de x_test sin incluir el id

In [43]:
x_test_text = ds_test["review_es"]

## Limpiar y Tokenizar el texto

In [36]:
def clean_text(text):
  return text.strip()

In [37]:
def tokenizer(text):
    #return [w for w in word_tokenize(text) if w.isalpha()] # si solo nos interesan palabras
    return word_tokenize(text)
tokenizer(clean_text(x_train_text.iloc[1]))[:10]

['Una',
 'pequeña',
 'pequeña',
 'producción.La',
 'técnica',
 'de',
 'filmación',
 'es',
 'muy',
 'incuestionable']

Defino un stop list de palabras en español

In [38]:
stoplist = stopwords.words("spanish")

In [39]:
stoplist[:15]

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con']

Tokenizamos las stopwords con el mismo tokenizador que utilizamos en el corpus

In [40]:
stoplist_tokenized = []
for w in stoplist:
    stoplist_tokenized = stoplist_tokenized + tokenizer(w)

In [42]:
# Armo una lista sin repeticiones
stoplist_tokenized = list(set(stoplist_tokenized))

## Bag of words

In [45]:
count_vect = CountVectorizer(preprocessor=clean_text,
                             tokenizer=tokenizer,
                             min_df=5,
                             stop_words=stoplist_tokenized)
X_train = count_vect.fit_transform(x_train_text) # cuenta frecuencia de tokens y define el diccionario
X_test = count_vect.transform(x_test_text) # cuenta frecuencia de tokens existentes en el diccionario
X_train

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<50000x59028 sparse matrix of type '<class 'numpy.int64'>'
	with 5448828 stored elements in Compressed Sparse Row format>

## Naive Bayes

In [50]:
clf = MultinomialNB(alpha=1) # alpha es el smoothing parameter
scores_cv_nb = cross_val_score(clf,
                               X_train, y_train,
                               cv = 10, scoring='f1_macro')
scores_cv_nb

array([0.8275768 , 0.83218872, 0.82135312, 0.84332887, 0.83459037,
       0.84366341, 0.84123539, 0.83502935, 0.83706416, 0.84219596])

In [51]:
# entreno el clasificador
clf = MultinomialNB(alpha=1).fit(X_train, y_train)

# predigo en el set de testeo
y_pred = clf.predict(X_test)

El formato esperado para el resultado es

In [55]:
sample.head()

,ID,sentimiento
0,60000,positivo
1,60001,positivo
2,60002,negativo
3,60003,positivo
4,60004,positivo


In [59]:
resultados = ds_test.copy()
resultados.drop(columns=["review_es"], inplace=True)
resultados['sentimiento'] = y_pred
resultados.head()

,ID,sentimiento
0,60000,negativo
1,60001,negativo
2,60002,negativo
3,60003,negativo
4,60004,negativo


In [66]:
total_positives = (resultados['sentimiento'] == "positivo").sum()
total_negatives = (resultados['sentimiento'] == "negativo").sum()
print(f"Positives: {total_positives}\nNegatives: {total_negatives}")

Positives: 4389
Negatives: 4210


# Guardado de resultados y modelo

In [69]:
if using_drive:
  resultados.to_csv(drive_path + "/Dataset/resultados.csv", index=False)
  joblib.dump(clf, drive_path + '/Models/naive_bayes.pkl')
else:
  resultados.to_csv("./Dataset/resultados.csv", index=False)
  joblib.dump(clf, './Models/naive_bayes.pkl')